In [1]:
import sys
print(sys.executable)
print(sys.version)

# Location of generated Protobuf files
sys.path.append('D:\lnd\proto\lnd\lnrpc')

C:\Users\Duncan\anaconda3\envs\myenv\python.exe
3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [2]:
import grpc
import os
from google.protobuf.json_format import MessageToDict
from lightning_pb2 import ForwardingHistoryRequest
from lightning_pb2 import ChannelGraphRequest
from lightning_pb2_grpc import LightningStub
import pandas as pd
import networkx as nx
import ipycytoscape

In [3]:
#import lightning_pb2 as ln
#import lightning_pb2_grpc as lngrpc

In [4]:
# Function to create a gRPC channel with the LND node
def create_grpc_channel(cert_file, macaroon_file):
    with open(cert_file, 'rb') as f:
        cert = f.read()
    with open(macaroon_file, 'rb') as f:
        macaroon = f.read().hex()

    credentials = grpc.ssl_channel_credentials(cert)
    channel = grpc.secure_channel('localhost:10009', credentials)
    return channel, macaroon


In [5]:
# Function to get node information
def get_node_info(channel, macaroon):
    stub = lngrpc.LightningStub(channel)
    request = ln.GetInfoRequest()
    metadata = [('macaroon', macaroon)]
    response = stub.GetInfo(request, metadata=metadata)
    return response

In [6]:
# Function to get forwarding history
def get_forwarding_history(stub, start_time, end_time, macaroon):
    # Create a request with the specified time range (in Unix timestamp)
    request = ForwardingHistoryRequest(start_time=start_time, end_time=end_time, num_max_events=10000)
    #request = ForwardingHistoryRequest(start_time=start_time, end_time=end_time)
    #request = ForwardingHistoryRequest()
    
    # Include the macaroon in the metadata
    metadata = [('macaroon', macaroon)]

    # Make the gRPC call
    response = stub.ForwardingHistory(request, metadata=metadata)
    
    # Convert response to dictionary
    events = [MessageToDict(event) for event in response.forwarding_events]
    
    # Return as pandas DataFrame
    return pd.DataFrame(events)


In [7]:
def get_network_graph(stub, macaroon):
    
    # Include the macaroon in the metadata
    metadata = [('macaroon', macaroon)]
    
    request = ChannelGraphRequest(include_unannounced=True)
    response = stub.DescribeGraph(request, metadata=metadata)
    return response

In [8]:
def create_node_id_to_alias_map(graph):
    node_id_to_alias = {}
    for node in graph.nodes:
        node_id_to_alias[node.pub_key] = node.alias
    return node_id_to_alias

In [9]:
def create_channel_id_to_node_ids_map(graph):
    channel_id_to_node_ids = {}
    for edge in graph.edges:
        channel_id_to_node_ids[str(edge.channel_id)] = (edge.node1_pub, edge.node2_pub)
    return channel_id_to_node_ids

In [10]:
def get_alias_for_channel(channel_id, channel_id_to_node_ids, node_id_to_alias):
    node_ids = channel_id_to_node_ids.get(str(channel_id), None)
    if node_ids:
        return node_id_to_alias.get(node_ids[0]), node_id_to_alias.get(node_ids[1])
    return None, None


In [11]:
# Load the RTL data from the CSV file
def load_data(filepath):
    try:
        # Attempt to read the CSV file
        return pd.read_csv(filepath, encoding='utf-8')
    except FileNotFoundError:
        # Handle the case where the file doesn't exist
        print(f"Error: The file {filepath} was not found.")
        return None
    except PermissionError:
        # Handle the case where the file exists but you don't have the permission to open it
        print(f"Error: Permission denied when trying to read the file {filepath}.")
        return None
    except pd.errors.EmptyDataError:
        # Handle the case where the file is empty
        print(f"Error: The file {filepath} is empty.")
        return None
    except pd.errors.ParserError:
        # Handle the case where the file has parsing issues
        print(f"Error: The file {filepath} does not appear to be a valid CSV or is improperly formatted.")
        return None
    except Exception as e:
        # Handle any other exception and print out a message
        print(f"An unexpected error occurred: {e}")
        return None

In [12]:
def add_nodes_and_edges(G, df):
    """Add nodes and edges to the graph G based on the dataframe df."""
    for index, row in df.iterrows():
        
        # Convert node names to strings to handle numerical aliases
        u = str(row['alias_in'])
        v = str(row['alias_out'])
        
        # Convert 'amt_out_msat' and 'fee_msat' to integers
        weight = int(row['amt_out_msat'])
        fee = int(row['fee_msat'])
        
        # Check if there's already an edge (i.e., a previous transaction between these channels)
        if G.has_edge(u, v):
            edge_data = G[u][v]
            edge_data['weight'] += weight
            edge_data['fees'].append(fee)
        else:
            G.add_edge(u, v, weight=weight, fees=[fee])
    
    # Average out the fees and compute total transactions
    total_transactions = sum(nx.get_edge_attributes(G, 'weight').values())
    for u, v, d in G.edges(data=True):
        d['avg_fee'] = sum(d['fees']) / len(d['fees'])
        d['normalized_fee'] = d['avg_fee'] / total_transactions * 10000

    return total_transactions


In [13]:
# Assign classes to nodes based on normalized transactions and fees
def assign_classes_to_nodes (cyto_graph, df):
    
    # Ensure numeric columns are of the correct type
    df['fee_msat'] = pd.to_numeric(df['fee_msat'], errors='coerce')
    df['amt_out_msat'] = pd.to_numeric(df['amt_out_msat'], errors='coerce')

    for node in cyto_graph.graph.nodes:
        node_alias = node.data['id']

        classes = []
        
        # # Calculate total fees and total transactions for each node
        total_fees = df.groupby('alias_out')['fee_msat'].sum()
        total_transactions_gr = df.groupby('alias_out')['amt_out_msat'].sum()

        max_transaction = total_transactions_gr.max()

        # Normalize the transactions
        normalized_transactions_node = (total_transactions_gr / max_transaction).to_dict()
        normalized_fees = (total_fees / total_transactions_gr * 10000).to_dict()

        # Assign fee classes
        if node_alias in normalized_fees:

            fee = normalized_fees[node_alias]
            if fee <= 0.1:
                classes.append('low_fee')
            elif fee <= 0.2:
                classes.append('medium_fee')
            else:
                classes.append('high_fee')
        else:
            classes.append('no_fee')

        # Assign transaction classes
        if node_alias in normalized_transactions_node:
            trans = normalized_transactions_node[node_alias]
            if trans <= 0.1:
                classes.append('tiny_transaction')
            elif trans <= 0.5:
                classes.append('small_transaction')
            elif trans <= 0.9:
                classes.append('medium_transaction')
            else:
                classes.append('large_transaction')

        # Combine classes with space
        node.classes = ' '.join(classes)

In [14]:
# Assign classes to edges based on normalized transaction volumes
def assign_classes_to_edges (cyto_graph, G, total_transactions):

    for edge in cyto_graph.graph.edges:

        u, v = edge.data['source'], edge.data['target']   
        d = G[u][v]

        normalized_transaction = d['weight']/total_transactions * 10000

        if (normalized_transaction < 1):
            edge.classes = 'tiny_volume'
        elif normalized_transaction < 10:
            edge.classes = 'small_volume'
        elif normalized_transaction < 100:
            edge.classes = 'medium_volume'
        else:
            edge.classes = 'large_volume'

In [15]:
# Set styles based on classes
def set_styles (cyto_graph):
    
    cyto_graph.set_style([
        {
            'selector': 'node.no_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'red'
            }
        },
        {
            'selector': 'node.low_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'orange'
            }
        },
        {
            'selector': 'node.medium_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'yellow'
            }
        },
        {
            'selector': 'node.high_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'green'
            }
        },
            {
            'selector': 'node.tiny_transaction',
            'style': {
                'width': '10px',
                'height': '10px'
            }
        },
        {
            'selector': 'node.small_transaction',
            'style': {
                'width': '20px',
                'height': '20px'
            }
        },
        {
            'selector': 'node.medium_transaction',
            'style': {
                'width': '30px',
                'height': '30px'
            }
        },
        {
            'selector': 'node.large_transaction',
            'style': {
                'width': '50px',
                'height': '50px'
            }
        },
        {
            'selector': 'edge.tiny_volume',
            'style': {
                'width': '0px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.small_volume',
            'style': {
                'width': '1px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.medium_volume',
            'style': {
                'width': '2px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.large_volume',
            'style': {
                'width': '3px',
                'line-color': 'black'
            }
        }
    ])


In [16]:
# Replace with your LND node's certificate, macaroon file path and server port
cert_file = r'D:\lnd\node1\tls.cert'
macaroon_file = r'D:\lnd\node1\data\chain\bitcoin\regtest\admin.macaroon'
rpcserver = 'localhost:10009'

channel, macaroon = create_grpc_channel(cert_file, macaroon_file)
#info = get_node_info(channel, macaroon)

#print(info)

#lnd = LNDClient(rpcserver, cert=tls_cert_path, macaroon_path=macaroon_path)
#from lndgrpc import LNDClient
#lnd = LNDClient(rpcserver, cert=cert_file, macaroon_path=macaroon_file)

#channel, macaroon = create_grpc_channel(cert_file, macaroon_file)
stub = LightningStub(channel)

start_time = 1609459200  # Example start time
end_time = 1800239131    # Example end time

# Fetch forwarding history
df = get_forwarding_history(stub, start_time, end_time, macaroon)

# Fetch the graph and create the maps
graph = get_network_graph(stub, macaroon)
node_id_to_alias = create_node_id_to_alias_map(graph)
channel_id_to_node_ids = create_channel_id_to_node_ids_map(graph)

# Update the DataFrame with aliases
df['alias_in'], df['alias_out'] = zip(*df.apply(lambda row: get_alias_for_channel(row['chanIdIn'], channel_id_to_node_ids, node_id_to_alias), axis=1))

# ALign the column names returned from gRPC with the CSV column names
df = df.rename(columns={
'feeMsat': 'fee_msat',
'amtInMsat': 'amt_in_msat',
'amtOutMsat': 'amt_out_msat',
'timestampNs': 'timestamp_ns'
})

# Create a new Cytograph
G = nx.Graph()

#Note, the provided 5 days of sample data has been anonymized for privacy. Using actual RTL node data will show the node ids
#df = load_data ("Forwarding-history-sample.csv")
#df = load_data ("Forwarding-history.csv")

# Check if the data was loaded successfully
if df is None:
    # If df is None, print an error message and exit
    print("Transaction data could not be loaded. Please check the file path and format or API connection.")
    sys.exit(1)

total_transactions = add_nodes_and_edges(G, df)

cyto_graph = ipycytoscape.CytoscapeWidget()
cyto_graph.graph.add_graph_from_networkx(G)

cyto_graph.set_layout(name='circle')

assign_classes_to_nodes (cyto_graph, df)
assign_classes_to_edges (cyto_graph, G, total_transactions)

set_styles (cyto_graph)

# Note: 
# Ensure that ipytoscape is enabled in Jupyter Lab or Jupyter Notebook, in a Conda shell run the following
# jupyter nbextension enable --py --sys-prefix ipycytoscape
# Start Jupyter with "jupyter notebook (or jupyter lab) --NotebookApp.iopub_data_rate_limit=1.0e10", in Powershell if datarate is exceeded
# This can happen with large numbers of transactions

cyto_graph

    

  

    
    


CytoscapeWidget(cytoscape_layout={'name': 'circle'}, cytoscape_style=[{'selector': 'node.no_fee', 'style': {'c…